In [1]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

# config_file = "../../../config/agam.yaml"
config_file = "../../../config/afun.yaml"

In [3]:
from bokeh.io import output_notebook
import malariagen_data
from IPython.display import Markdown
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Methods

## Data sources

This site analyses genome variation data from the {term}`Malaria Vector Genome Observatory`. 
For a complete list of the sample sets used in the current analysis version, with information 
about the corresponding contributors, data releases and citations, please see the 
{doc}`data-sources` page.

## Data retrieval

In [7]:
Markdown(f"""
Data were retrieved from the {{term}}`Malaria Vector Genome Observatory` cloud data repository 
hosted in Google Cloud Storage (GCS) via the {{term}}`MalariaGEN Python API` version 
{malariagen_data.__version__}. For further information about the configuration of 
""")


Data were retrieved from the {term}`Malaria Vector Genome Observatory` cloud data repository 
hosted in Google Cloud Storage (GCS) via the {term}`MalariaGEN Python API` version 
15.0.1. For further information about the configuration of 
